In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Fetching necessary files

!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Validation_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Validation_GroundTruth.zip

!unzip ./ISIC2018_Task1-2_Training_Input.zip
# !rm ./ISIC2018_Task1-2_Training_Input.zip
!unzip ./ISIC2018_Task1_Training_GroundTruth.zip
# !rm ./ISIC2018_Task1_Training_GroundTruth.zip
!unzip ./ISIC2018_Task1-2_Validation_Input.zip
# !rm ./ISIC2018_Task1-2_Validation_Input.zip
!unzip ./ISIC2018_Task1_Validation_GroundTruth.zip
# !rm ./ISIC2018_Task1_Validation_GroundTruth.zip
!mkdir ./Preproc

In [2]:
import os
from tqdm import tqdm
import torchinfo
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split
import torchvision
import cv2 as cv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
from torchvision.io import read_image
# import scipy as sc

In [3]:
os.chdir('E:\\Segmentation')

TRAIN_INPUT_DIR = 'ISIC2018_Task1-2_Training_Input/'
TRAIN_GT_DIR = 'ISIC2018_Task1_Training_GroundTruth/'
TRAIN_INTERM_DIR = 'ISIC2017-Training-Interm'

VAL_INPUT_DIR = 'ISIC2018_Task1-2_Validation_Input/'
VAL_GT_DIR = 'ISIC2018_Task1_Validation_GroundTruth/'
VAL_INTERM_DIR = 'ISIC2017-Validation-Output'

BATCH_SIZE = 4
LEARNING_RATE = 1e-4
EPOCHS = 100
IM_H, IM_W = 256, 256
TRAINING_NOISE = 0
DROPOUT = .5

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
print(DEVICE)

cuda


In [5]:
def centroid(img, lcc=False):
  if lcc:
    img = img.astype(np.uint8)
    nb_components, output, stats, centroids = cv.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    if len(sizes) > 2:
      max_label = 1
      max_size = sizes[1]

      for i in range(2, nb_components):
          if sizes[i] > max_size:
              max_label = i
              max_size = sizes[i]

      img2 = np.zeros(output.shape)
      img2[output == max_label] = 255
      img = img2

  if len(img.shape) > 2:
    M = cv.moments(img[:,:,1])
  else:
    M = cv.moments(img)

  if M["m00"] == 0:
    return (img.shape[0] // 2, img.shape[1] // 2)
  
  cX = int(M["m10"] / M["m00"])
  cY = int(M["m01"] / M["m00"])
  return (cX, cY)

def to_polar(input_img, center):
  input_img = input_img.astype(np.float32)
  value = np.sqrt(((input_img.shape[0]/2.0)**2.0)+((input_img.shape[1]/2.0)**2.0))
  polar_image = cv.linearPolar(input_img, center, value, cv.WARP_FILL_OUTLIERS)
  return polar_image

def to_cart(input_img, center):
  input_img = input_img.astype(np.float32)
  value = np.sqrt(((input_img.shape[1]/2.0)**2.0)+((input_img.shape[0]/2.0)**2.0))
  polar_image = cv.linearPolar(input_img, center, value, cv.WARP_FILL_OUTLIERS + cv.WARP_INVERSE_MAP)
  polar_image = polar_image.astype(np.uint8)
  return polar_image

def calc_dice(input_img, target):
    tp = np.sum(np.minimum(input_img, target))
    fp = np.sum(np.minimum(input_img, 1 - target))
    fn = np.sum(np.minimum(1 - input_img, target))
    return 2 * tp / (2 * tp + fp + fn)

In [6]:
class ISICDataset(torch.utils.data.Dataset):
    def __init__(self, input_folder, seg_folder, transform=None, target_transform=None):
        self.in_files = glob(os.path.join(input_folder, '*.jpg'))
        self.gt_files = glob(os.path.join(seg_folder, '*.png'))
        df = pd.read_csv('centers_new.csv')
        self.centers = dict()
        for item in df['file']:
            self.centers[item] = df[df['file'] == item].to_numpy()[0,1:3].tolist()
        self.tfy = torchvision.transforms.Resize((384, 512), interpolation=torchvision.transforms.InterpolationMode.BICUBIC)
        self.tfX = torchvision.transforms.Resize((IM_W, IM_H), interpolation=torchvision.transforms.InterpolationMode.BICUBIC)
        print(len(self.in_files), len(self.gt_files))
        

    def __len__(self):
        return len(self.in_files)

    def __getitem__(self, idx):
        in_img = self.tfX(torchvision.io.read_image(self.in_files[idx], torchvision.io.image.ImageReadMode.RGB)).float()
        in_img = (in_img - torch.mean(in_img, dim=(1,2),keepdim=True)) / torch.std(in_img, dim=(1,2), keepdim=True)
        gt_img = self.tfy(torchvision.io.read_image(self.gt_files[idx], torchvision.io.image.ImageReadMode.GRAY)).float()
        gt_img = gt_img / torch.max(gt_img)
        # gt_img = torch.unsqueeze(gt_img, 0)
        X = in_img
        filename = os.path.split(self.in_files[idx])[-1][:-4]
        center = self.centers[self.in_files[idx]]        
        y = gt_img
        mask_l = torch.ones(1, 384, 512, dtype=torch.float32)
        mask_s = torch.ones(1, IM_H, IM_W, dtype=torch.float32)
        return (X, mask_l, mask_s, y, filename, center)

In [7]:
class ISICDatasetTrain(torch.utils.data.Dataset):
    def __init__(self, src):
        self.src = src
        self.tf = torchvision.transforms.Resize((IM_W, IM_H))

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        X, mask_l, mask_s, y, filename, center = self.src[idx]
        y = self.tf(y)
        y = torch.round(y)
        return (X, mask_l, mask_s, y, filename, center)

In [8]:
class ISICDatasetTest(torch.utils.data.Dataset):
    def __init__(self, src):
        self.src = src

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        X, mask_l, mask_s, y, filename, center = self.src[idx]
        y = torch.round(y)
        return (X, mask_l, mask_s, y, filename, center)

In [9]:
full_ds = ISICDataset(TRAIN_INPUT_DIR, TRAIN_GT_DIR)
n, nt = len(full_ds), int(len(full_ds) / 10)
train_ds_p, valid_ds_p, test_ds_p = random_split(full_ds, [n - 2 * nt, nt, nt])
train_ds = ISICDatasetTrain(train_ds_p)
valid_ds = ISICDatasetTrain(valid_ds_p)
test_ds = ISICDatasetTest(test_ds_p)

2594 2594


In [10]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False, pin_memory=False)

In [11]:
# with mask

def train(model):
    num_batches = len(train_dl)
    model.train()
    losses = []
    accuracies = []
    dices = []
    with tqdm(total=num_batches) as pbar:
        pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
        for (X, mask_l, mask_s, y, filename, center) in train_dl:
            loss, met = model.fit(X, mask_s, y)
            losses.append(loss)
            accuracies.append((met[0] + met[3]) / (met[0] + met[1] + met[2] + met[3]))
            dices.append(met[0] * 2 / (met[0] * 2 + met[1] + met[2]))
            pbar.update(1)
            pbar.set_description(f"Avg. Loss: {torch.mean(torch.tensor(losses)):.4f}, Avg. Accuracy: {torch.mean(torch.tensor(accuracies)):.4f}, Avg. Dice: {torch.mean(torch.tensor(dices)):.4f}")
    return torch.mean(torch.tensor(losses)).item(), torch.mean(torch.tensor(dices)).item()

def test(model):
    num_batches = len(valid_dl)
    model.eval()
    losses = []
    accuracies = []
    dices = []
    with tqdm(total=num_batches) as pbar:
        pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
        with torch.no_grad():
            for (X, mask_l, mask_s, y, filename, center) in valid_dl:
                loss, met = model.test(X, mask_s, y)
                losses.append(loss)
                accuracies.append((met[0] + met[3]) / (met[0] + met[1] + met[2] + met[3]))
                dices.append(met[0] * 2 / (met[0] * 2 + met[1] + met[2]))
                pbar.update(1)
                pbar.set_description(f"Avg. Loss: {torch.mean(torch.tensor(losses)):.4f}, Avg. Accuracy: {torch.mean(torch.tensor(accuracies)):.4f}, Avg. Dice: {torch.mean(torch.tensor(dices)):.4f}")
    return torch.mean(torch.tensor(losses)).item(), torch.mean(torch.tensor(dices)).item()

# Loss functions

In [32]:
def iou_loss(pred, mask):
    weit = 1 + 5 * torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    
    pred = torch.sigmoid(pred)
    inter = ((pred * mask) * weit).sum(dim=(2, 3))
    union = ((pred + mask) * weit).sum(dim=(2, 3))
    iou = 1 - (inter + 1) / (union - inter + 1)

    return iou.mean()


def structure_loss(pred, mask):
    weit = 1 + 5 * torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wbce = F.binary_cross_entropy_with_logits(pred, mask, reduce='none')
    wbce = (weit * wbce).sum(dim=(2, 3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask) * weit).sum(dim=(2, 3))
    union = ((pred + mask) * weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1) / (union - inter + 1)

    return (wbce + wiou).mean()

def dice_loss(pred, mask, cover):
    weit = 1 # + 5 * torch.abs(F.avg_pool2d(mask, kernel_size=7, stride=1, padding=3) - mask)
    
    pred = torch.sigmoid(pred)
    tp = ((pred * mask) * weit).sum(dim=(2, 3))
    fp = ((pred * (1 - mask)) * weit).sum(dim=(2, 3))
    fn = (((1 - pred) * mask) * weit).sum(dim=(2, 3))
    dice = 1 - (2 * tp + 1) / (2 * tp + fp + fn + 1)

    return dice.mean()

class FocalLoss():
    def __init__(self, gamma):
        self.gamma = gamma
        
    def __call__(self, pred, mask):
        pred = torch.sigmoid(pred)
        dist = torch.abs(pred - mask)
        bce = -torch.log((1 - dist) + 1e-15)
        focal_adjusted = torch.mean(dist**self.gamma * bce)
        return focal_adjusted

def focal_struct_loss(pred, mask):
    weit = torch.abs(mask - F.sigmoid(pred))
    wbce = F.binary_cross_entropy_with_logits(pred, mask, reduce='none')
    wbce = (weit * wbce).sum(dim=(2, 3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask) * weit).sum(dim=(2, 3))
    union = ((pred + mask) * weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1) / (union - inter + 1)

    return (wbce + wiou).mean()

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from lib.pvtv2 import pvt_v2_b2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
# from mmcv.cnn import ConvModule
from torch.nn import Conv2d, UpsamplingBilinear2d
import warnings
import torch
# from mmcv.cnn import constant_init, kaiming_init
from torch import nn
from torchvision.transforms.functional import normalize
from PVTv2.pvtv2 import pvt_v2_b2
warnings.filterwarnings('ignore')

class SpatBlock(nn.Module):
    def __init__(self, dim=32, order=5):
        super().__init__()
        self.dim = dim
        self.order = order
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.blocks = nn.ModuleList([nn.Conv2d(1, dim, 1)] + [nn.Conv2d(1, dim, 1, bias=False) for _ in range(order-1)])

        self = self.to(self.device)

    def forward(self, x):
        B, C, H, W = x.shape

        pos = torch.arange(W) / W
        pos = torch.reshape(pos, (1, 1, 1, W)).to(self.device)
        
        func = torch.zeros_like(x)

        for i, block in enumerate(self.blocks):
            func += block(pos**(i+1))

        out = x + func
        
        return out
        

def structure_loss(pred, mask):
    weit = 1 + 5 * torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wbce = F.binary_cross_entropy_with_logits(pred, mask, reduce='none')
    wbce = (weit * wbce).sum(dim=(2, 3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask) * weit).sum(dim=(2, 3))
    union = ((pred + mask) * weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1) / (union - inter + 1)

    return (wbce + wiou).mean()

# def structure_loss(pred, mask):
#     pred = torch.sigmoid(pred)
#     loss1 = F.binary_cross_entropy(pred, mask)
#     tp = pred * mask
#     fp = pred * (1 - mask)
#     fn = (1 - pred) * mask
#     tp = torch.mean(tp, dim=0)
#     fp = torch.mean(fp, dim=0)
#     fn = torch.mean(fn, dim=0)
#     iou = tp / (tp + fp + fn)
#     iou = torch.mean(iou)
#     loss2 = 1 - iou
#     return loss1 + loss2

class BasicConv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1):
        super(BasicConv2d, self).__init__()

        self.conv = nn.Conv2d(in_planes, out_planes,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False)
        self.bn = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
        
class Block(nn.Sequential):
    def __init__(self, input_num, num1, num2, dilation_rate, drop_out, bn_start=True, norm_layer=nn.BatchNorm2d):
        super(Block, self).__init__()
        if bn_start:
            self.add_module('norm1', norm_layer(input_num)),

        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(in_channels=input_num, out_channels=num1, kernel_size=1)),

        self.add_module('norm2', norm_layer(num1)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(in_channels=num1, out_channels=num2, kernel_size=3,
                                            dilation=dilation_rate, padding=dilation_rate)),
        self.drop_rate = drop_out

    def forward(self, _input):
        feature = super(Block, self).forward(_input)
        if self.drop_rate > 0:
            feature = F.dropout2d(feature, p=self.drop_rate, training=self.training)
        return feature


def Upsample(x, size, align_corners = False):
    """
    Wrapper Around the Upsample Call
    """
    return nn.functional.interpolate(x, size=size, mode='bilinear', align_corners=align_corners)

    
# def last_zero_init(m):
#     if isinstance(m, nn.Sequential):
#         constant_init(m[-1], val=0)
#     else:
#         constant_init(m, val=0)


class ContextBlock(nn.Module):

    def __init__(self,
                 inplanes,
                 ratio,
                 pooling_type='att',
                 fusion_types=('channel_mul', )):
        super(ContextBlock, self).__init__()
        assert pooling_type in ['avg', 'att']
        assert isinstance(fusion_types, (list, tuple))
        valid_fusion_types = ['channel_add', 'channel_mul']
        assert all([f in valid_fusion_types for f in fusion_types])
        assert len(fusion_types) > 0, 'at least one fusion should be used'
        self.inplanes = inplanes
        self.ratio = ratio
        self.planes = int(inplanes * ratio)
        self.pooling_type = pooling_type
        self.fusion_types = fusion_types
        if pooling_type == 'att':
            self.conv_mask = nn.Conv2d(inplanes, 1, kernel_size=1)
            self.softmax = nn.Softmax(dim=2)
        else:
            self.avg_pool = nn.AdaptiveAvgPool2d(1)
        if 'channel_add' in fusion_types:
            self.channel_add_conv = nn.Sequential(
                nn.Conv2d(self.inplanes, self.planes, kernel_size=1),
                nn.LayerNorm([self.planes, 1, 1]),
                nn.ReLU(inplace=True),  # yapf: disable
                nn.Conv2d(self.planes, self.inplanes, kernel_size=1))
        else:
            self.channel_add_conv = None
        if 'channel_mul' in fusion_types:
            self.channel_mul_conv = nn.Sequential(
                nn.Conv2d(self.inplanes, self.planes, kernel_size=1),
                nn.LayerNorm([self.planes, 1, 1]),
                nn.ReLU(inplace=True),  # yapf: disable
                nn.Conv2d(self.planes, self.inplanes, kernel_size=1))
        else:
            self.channel_mul_conv = None

    def spatial_pool(self, x):
        batch, channel, height, width = x.size()
        if self.pooling_type == 'att':
            input_x = x
            # [N, C, H * W]
            input_x = input_x.view(batch, channel, height * width)
            # [N, 1, C, H * W]
            input_x = input_x.unsqueeze(1)
            # [N, 1, H, W]
            context_mask = self.conv_mask(x)
            # [N, 1, H * W]
            context_mask = context_mask.view(batch, 1, height * width)
            # [N, 1, H * W]
            context_mask = self.softmax(context_mask)
            # [N, 1, H * W, 1]
            context_mask = context_mask.unsqueeze(-1)
            # [N, 1, C, 1]
            context = torch.matmul(input_x, context_mask)
            # [N, C, 1, 1]
            context = context.view(batch, channel, 1, 1)
        else:
            # [N, C, 1, 1]
            context = self.avg_pool(x)

        return context

    def forward(self, x):
        # [N, C, 1, 1]
        context = self.spatial_pool(x)

        out = x
        if self.channel_mul_conv is not None:
            # [N, C, 1, 1]
            channel_mul_term = torch.sigmoid(self.channel_mul_conv(context))
            out = out + out * channel_mul_term
        if self.channel_add_conv is not None:
            # [N, C, 1, 1]
            channel_add_term = self.channel_add_conv(context)
            out = out + channel_add_term

        return out



class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1   = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)


class ConvBranch(nn.Module):
    def __init__(self, in_features, hidden_features = None, out_features = None):
        super().__init__()
        hidden_features = hidden_features or in_features
        out_features = out_features or in_features
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_features, hidden_features, 1, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 3, padding=1, groups=hidden_features, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 1, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 3, padding=1, groups=hidden_features, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 1, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.SiLU(inplace=True)
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 3, padding=1, groups=hidden_features, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv7 = nn.Sequential(
            nn.Conv2d(hidden_features, out_features, 1, bias=False),
            nn.ReLU(inplace=True)
        )
        self.ca = ChannelAttention(64)
        self.sa = SpatialAttention()
        self.sigmoid_spatial = nn.Sigmoid()
    
    def forward(self, x):
        res1 = x
        res2 = x
        x = self.conv1(x)        
        x = x + self.conv2(x)
        x = self.conv3(x)
        x = x + self.conv4(x)
        x = self.conv5(x)
        x = x + self.conv6(x)
        x = self.conv7(x)
        x_mask = self.sigmoid_spatial(x)
        res1 = res1 * x_mask
        return res2 + res1

              
class GLSA(nn.Module):

    def __init__(self, input_dim=512, embed_dim=32, k_s=3):
        super().__init__()
                      
        self.conv1_1 = BasicConv2d(embed_dim*2,embed_dim, 1)
        self.conv1_1_1 = BasicConv2d(input_dim//2,embed_dim,1)
        self.local_11conv = nn.Conv2d(input_dim//2,embed_dim,1)
        self.global_11conv = nn.Conv2d(input_dim//2,embed_dim,1)
        self.GlobelBlock = ContextBlock(inplanes= embed_dim, ratio=2)
        self.local = ConvBranch(in_features = embed_dim, hidden_features = embed_dim, out_features = embed_dim)

    def forward(self, x):
        b, c, h, w = x.size()
        x_0, x_1 = x.chunk(2,dim = 1)  
        
    # local block 
        local = self.local(self.local_11conv(x_0))
        
    # Globel block    
        Globel = self.GlobelBlock(self.global_11conv(x_1))

    # concat Globel + local
        x = torch.cat([local,Globel], dim=1)
        x = self.conv1_1(x)

        return x    

class SBA(nn.Module):

    def __init__(self,input_dim = 64):
        super().__init__()

        self.input_dim = input_dim

        self.d_in1 = BasicConv2d(input_dim//2, input_dim//2, 1)
        self.d_in2 = BasicConv2d(input_dim//2, input_dim//2, 1)       
                

        self.conv = nn.Sequential(BasicConv2d(input_dim, input_dim, 3,1,1), nn.Conv2d(input_dim, 1, kernel_size=1, bias=False))
        self.fc1 = nn.Conv2d(input_dim, input_dim//2, kernel_size=1, bias=False)
        self.fc2 = nn.Conv2d(input_dim, input_dim//2, kernel_size=1, bias=False)
        
        self.Sigmoid = nn.Sigmoid()
        
    def forward(self, H_feature, L_feature):

        L_feature = self.fc1(L_feature)
        H_feature = self.fc2(H_feature)
        
        g_L_feature =  self.Sigmoid(L_feature)
        g_H_feature = self.Sigmoid(H_feature)
        
        L_feature = self.d_in1(L_feature)
        H_feature = self.d_in2(H_feature)


        L_feature = L_feature + L_feature * g_L_feature + (1 - g_L_feature) * Upsample(g_H_feature * H_feature, size= L_feature.size()[2:], align_corners=False)
        H_feature = H_feature + H_feature * g_H_feature + (1 - g_H_feature) * Upsample(g_L_feature * L_feature, size= H_feature.size()[2:], align_corners=False) 
        
        H_feature = Upsample(H_feature, size = L_feature.size()[2:])
        out = self.conv(torch.cat([H_feature,L_feature], dim=1))
        return out
        
            
class DuAT(nn.Module):
    def __init__(self, dim=32, dims= [64, 256, 512, 1024], learning_rate=None,  loss_fn=None, optimizer=None, device=None, weight_decay=None):
        super(DuAT, self).__init__()

                    
        if (device is None):
            self.device = DEVICE
        else:
            self.device = device

        if (learning_rate is None):
            self.learning_rate = 1e-5
        else:
            self.learning_rate = learning_rate
        
        if (weight_decay is None):
            self.weight_decay = 1e-5
        else:
            self.weight_decay = weight_decay

        self.dims = dims

        # self.pad = False
        resnet50 = torchvision.models.resnet50(weights="DEFAULT")
        self.e1 = nn.Sequential(resnet50._modules['conv1'], resnet50._modules['bn1'], resnet50._modules['relu'])
        self.e2 = nn.Sequential(resnet50._modules['maxpool'], resnet50._modules['layer1'])
        self.e3 = nn.Sequential(resnet50._modules['layer2'])
        self.e4 = nn.Sequential(resnet50._modules['layer3'])
        
        c1_in_channels, c2_in_channels, c3_in_channels, c4_in_channels = dims[0], dims[1], dims[2], dims[3]

        self.shuffle4 = nn.Conv2d(c4_in_channels, c4_in_channels, 1)
        self.shuffle3 = nn.Conv2d(c3_in_channels, c3_in_channels, 1)
        self.shuffle2 = nn.Conv2d(c2_in_channels, c2_in_channels, 1)
        self.shuffle1 = nn.Conv2d(c1_in_channels, c1_in_channels, 1)

        self.bn4 = nn.BatchNorm2d(c4_in_channels)
        self.bn3 = nn.BatchNorm2d(c3_in_channels)
        self.bn2 = nn.BatchNorm2d(c2_in_channels)
        self.bn1 = nn.BatchNorm2d(c1_in_channels)

        self.embed4 = SpatBlock(dim=c4_in_channels)
        self.embed3 = SpatBlock(dim=c3_in_channels)
        self.embed2 = SpatBlock(dim=c2_in_channels)
        self.embed1 = SpatBlock(dim=c1_in_channels)

        self.bn4 = nn.BatchNorm2d(c4_in_channels)
        self.bn3 = nn.BatchNorm2d(c3_in_channels)
        self.bn2 = nn.BatchNorm2d(c2_in_channels)
        self.bn1 = nn.BatchNorm2d(c1_in_channels)

        self.GLSA_c4 = GLSA(input_dim=c4_in_channels, embed_dim=dim)
        self.GLSA_c3 = GLSA(input_dim=c3_in_channels, embed_dim=dim)
        self.GLSA_c2 = GLSA(input_dim=c2_in_channels, embed_dim=dim)
        self.L_feature = BasicConv2d(c1_in_channels, dim, 3,1,1)
        
        self.SBA = SBA(input_dim = dim)
        self.fuse = BasicConv2d(dim * 2, dim, 1)
        self.fuse2 = nn.Sequential(BasicConv2d(dim*3, dim, 1,1),nn.BatchNorm2d(dim),nn.Conv2d(dim, 1, kernel_size=1, bias=False))


        for param in self.e1.parameters():
            param.requires_grad_(False)
            
        for param in self.e2.parameters():
            param.requires_grad_(False)
            
        for param in self.e3.parameters():
            param.requires_grad_(False)
            
        for param in self.e4.parameters():
            param.requires_grad_(False)

        if (loss_fn is None):
            self.loss_fn = structure_loss
        else:
            self.loss_fn = loss_fn

        if (optimizer is None):
            self.optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        else:
            self.optimizer = optimizer(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)

        self = self.to(self.device)

    def forward(self, x, mask):
        # backbone
        x = x.to(self.device)
        c1 = self.e1(x)
        c2 = self.e2(c1)
        c3 = self.e3(c2)
        c4 = self.e4(c3)


        
        # c1 = self.bn1(self.shuffle1(c1))
        # c2 = self.bn2(self.shuffle2(c2))
        # c3 = self.bn3(self.shuffle3(c3))
        # c4 = self.bn4(self.shuffle4(c4))
        
        # c1 = torch.cat([c1,
        #                 torch.arange(c1.shape[2], dtype=c1.dtype, device=self.device).view(1,1,-1,1).repeat(c1.shape[0], 1, 1, c1.shape[3]) / c1.shape[2],
        #                 torch.arange(c1.shape[3], dtype=c1.dtype, device=self.device).view(1,1,1,-1).repeat(c1.shape[0], 1, c1.shape[2], 1) / c1.shape[3]], dim=1)
        # c2 = torch.cat([c2,
        #                 torch.arange(c2.shape[2], dtype=c2.dtype, device=self.device).view(1,1,-1,1).repeat(c2.shape[0], 1, 1, c2.shape[3]) / c2.shape[2],
        #                 torch.arange(c2.shape[3], dtype=c2.dtype, device=self.device).view(1,1,1,-1).repeat(c2.shape[0], 1, c2.shape[2], 1) / c2.shape[3]], dim=1)
        # c3 = torch.cat([c3,
        #                 torch.arange(c3.shape[2], dtype=c3.dtype, device=self.device).view(1,1,-1,1).repeat(c1.shape[0], 1, 1, c3.shape[3]) / c3.shape[2],
        #                 torch.arange(c3.shape[3], dtype=c3.dtype, device=self.device).view(1,1,1,-1).repeat(c1.shape[0], 1, c3.shape[2], 1) / c3.shape[3]], dim=1)
        # c4 = torch.cat([c4,
        #                 torch.arange(c4.shape[2], dtype=c4.dtype, device=self.device).view(1,1,-1,1).repeat(c4.shape[0], 1, 1, c4.shape[3]) / c4.shape[2],
        #                 torch.arange(c4.shape[3], dtype=c4.dtype, device=self.device).view(1,1,1,-1).repeat(c4.shape[0], 1, c4.shape[2], 1) / c4.shape[3]], dim=1)

        c1 = self.bn1(self.embed1(c1))
        c2 = self.bn2(self.embed2(c2))
        c3 = self.bn3(self.embed3(c3))
        c4 = self.bn4(self.embed4(c4))
        
        n, _, h, w = c4.shape        
        _c4 = self.GLSA_c4(c4) # [1, 64, 11, 11]
        _c4 = Upsample(_c4, c3.size()[2:])
        _c3 = self.GLSA_c3(c3) # [1, 64, 22, 22]
        _c2 = self.GLSA_c2(c2) # [1, 64, 44, 44]
        
        output = torch.cat([Upsample(_c4, c2.size()[2:]), Upsample(_c3, c2.size()[2:]), _c2], dim=1)
        
        L_feature = self.L_feature(c1)  # [1, 64, 88, 88]
        H_feature = self.fuse(torch.cat([_c4, _c3], dim=1))
        H_feature = Upsample(H_feature,c2.size()[2:])
        
        output2 = self.SBA(H_feature,L_feature)
        h = x.shape[2] // 4
        output = F.interpolate(output, scale_factor=4, mode='bicubic')
        # output = torch.cat([output, Upsample(c1, output.size()[2:])], dim=1)
        output = self.fuse2(output)
        # return torch.sigmoid(output)
        output2 = F.interpolate(output2, scale_factor=2, mode='bicubic')

        
        return output, output2
        # return F.sigmoid(output[:,:,h:-h,:])
        
    def fit(self, X, mask, y):
        X, mask, y = X.to(self.device), mask.to(self.device), y.to(self.device)
        h1, h2 = self(X, mask)
        self.optimizer.zero_grad()
        loss = self.loss_fn(h1, y) + self.loss_fn(h2, y)
        loss.backward()
        self.optimizer.step()
        loss = loss.item()
        pred = (h2 > 0).int()
        numt = torch.sum(mask)
        TP = torch.sum(torch.minimum(y, pred)).item() / numt
        TN = torch.sum(torch.minimum(1-y, 1-pred) * mask).item() / numt
        FN = torch.sum(torch.minimum(y, 1-pred)).item() / numt
        FP = torch.sum(torch.minimum(1-y, pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))

    def test(self, X, mask, y):
        X, mask, y = X.to(self.device), mask.to(self.device), y.to(self.device)
        h1, h2 = self(X, mask)
        loss = self.loss_fn(h1, y) + self.loss_fn(h2, y)
        loss = loss.item()
        pred = (h2 > 0).int()
        numt = torch.sum(mask)
        TP = torch.sum(torch.minimum(y, pred)).item() / numt
        TN = torch.sum(torch.minimum(1-y, 1-pred) * mask).item() / numt
        FN = torch.sum(torch.minimum(y, 1-pred)).item() / numt
        FP = torch.sum(torch.minimum(1-y, pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))


# PVT DuAT

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from lib.pvtv2 import pvt_v2_b2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
# from mmcv.cnn import ConvModule
from torch.nn import Conv2d, UpsamplingBilinear2d
import warnings
import torch
# from mmcv.cnn import constant_init, kaiming_init
from torch import nn
from torchvision.transforms.functional import normalize
from BaseModel import BaseModel
from PVT.segmentation.pvt import pvt_large, pvt_medium, pvt_small, pvt_tiny
from PVTv2.pvtv2 import *
warnings.filterwarnings('ignore')

def structure_loss(pred, mask):
    weit = 1 + 5 * torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wbce = F.binary_cross_entropy_with_logits(pred, mask, reduce='none')
    wbce = (weit * wbce).sum(dim=(2, 3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask) * weit).sum(dim=(2, 3))
    union = ((pred + mask) * weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1) / (union - inter + 1)

    return (wbce + wiou).mean()

# def structure_loss(pred, mask):
#     pred = torch.sigmoid(pred)
#     loss1 = F.binary_cross_entropy(pred, mask)
#     tp = pred * mask
#     fp = pred * (1 - mask)
#     fn = (1 - pred) * mask
#     tp = torch.mean(tp, dim=0)
#     fp = torch.mean(fp, dim=0)
#     fn = torch.mean(fn, dim=0)
#     iou = tp / (tp + fp + fn)
#     iou = torch.mean(iou)
#     loss2 = 1 - iou
#     return loss1 + loss2

class BasicConv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1):
        super(BasicConv2d, self).__init__()

        self.conv = nn.Conv2d(in_planes, out_planes,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False)
        self.bn = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
        
class Block(nn.Sequential):
    def __init__(self, input_num, num1, num2, dilation_rate, drop_out, bn_start=True, norm_layer=nn.BatchNorm2d):
        super(Block, self).__init__()
        if bn_start:
            self.add_module('norm1', norm_layer(input_num)),

        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(in_channels=input_num, out_channels=num1, kernel_size=1)),

        self.add_module('norm2', norm_layer(num1)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(in_channels=num1, out_channels=num2, kernel_size=3,
                                            dilation=dilation_rate, padding=dilation_rate)),
        self.drop_rate = drop_out

    def forward(self, _input):
        feature = super(Block, self).forward(_input)
        if self.drop_rate > 0:
            feature = F.dropout2d(feature, p=self.drop_rate, training=self.training)
        return feature


def Upsample(x, size, align_corners = False):
    """
    Wrapper Around the Upsample Call
    """
    return nn.functional.interpolate(x, size=size, mode='bilinear', align_corners=align_corners)

    
# def last_zero_init(m):
#     if isinstance(m, nn.Sequential):
#         constant_init(m[-1], val=0)
#     else:
#         constant_init(m, val=0)


class ContextBlock(nn.Module):

    def __init__(self,
                 inplanes,
                 ratio,
                 pooling_type='att',
                 fusion_types=('channel_mul', )):
        super(ContextBlock, self).__init__()
        assert pooling_type in ['avg', 'att']
        assert isinstance(fusion_types, (list, tuple))
        valid_fusion_types = ['channel_add', 'channel_mul']
        assert all([f in valid_fusion_types for f in fusion_types])
        assert len(fusion_types) > 0, 'at least one fusion should be used'
        self.inplanes = inplanes
        self.ratio = ratio
        self.planes = int(inplanes * ratio)
        self.pooling_type = pooling_type
        self.fusion_types = fusion_types
        if pooling_type == 'att':
            self.conv_mask = nn.Conv2d(inplanes, 1, kernel_size=1)
            self.softmax = nn.Softmax(dim=2)
        else:
            self.avg_pool = nn.AdaptiveAvgPool2d(1)
        if 'channel_add' in fusion_types:
            self.channel_add_conv = nn.Sequential(
                nn.Conv2d(self.inplanes, self.planes, kernel_size=1),
                nn.LayerNorm([self.planes, 1, 1]),
                nn.ReLU(inplace=True),  # yapf: disable
                nn.Conv2d(self.planes, self.inplanes, kernel_size=1))
        else:
            self.channel_add_conv = None
        if 'channel_mul' in fusion_types:
            self.channel_mul_conv = nn.Sequential(
                nn.Conv2d(self.inplanes, self.planes, kernel_size=1),
                nn.LayerNorm([self.planes, 1, 1]),
                nn.ReLU(inplace=True),  # yapf: disable
                nn.Conv2d(self.planes, self.inplanes, kernel_size=1))
        else:
            self.channel_mul_conv = None

    def spatial_pool(self, x):
        batch, channel, height, width = x.size()
        if self.pooling_type == 'att':
            input_x = x
            # [N, C, H * W]
            input_x = input_x.view(batch, channel, height * width)
            # [N, 1, C, H * W]
            input_x = input_x.unsqueeze(1)
            # [N, 1, H, W]
            context_mask = self.conv_mask(x)
            # [N, 1, H * W]
            context_mask = context_mask.view(batch, 1, height * width)
            # [N, 1, H * W]
            context_mask = self.softmax(context_mask)
            # [N, 1, H * W, 1]
            context_mask = context_mask.unsqueeze(-1)
            # [N, 1, C, 1]
            context = torch.matmul(input_x, context_mask)
            # [N, C, 1, 1]
            context = context.view(batch, channel, 1, 1)
        else:
            # [N, C, 1, 1]
            context = self.avg_pool(x)

        return context

    def forward(self, x):
        # [N, C, 1, 1]
        context = self.spatial_pool(x)

        out = x
        if self.channel_mul_conv is not None:
            # [N, C, 1, 1]
            channel_mul_term = torch.sigmoid(self.channel_mul_conv(context))
            out = out + out * channel_mul_term
        if self.channel_add_conv is not None:
            # [N, C, 1, 1]
            channel_add_term = self.channel_add_conv(context)
            out = out + channel_add_term

        return out



class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1   = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)


class ConvBranch(nn.Module):
    def __init__(self, in_features, hidden_features = None, out_features = None):
        super().__init__()
        hidden_features = hidden_features or in_features
        out_features = out_features or in_features
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_features, hidden_features, 1, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 3, padding=1, groups=hidden_features, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 1, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 3, padding=1, groups=hidden_features, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 1, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.SiLU(inplace=True)
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 3, padding=1, groups=hidden_features, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv7 = nn.Sequential(
            nn.Conv2d(hidden_features, out_features, 1, bias=False),
            nn.ReLU(inplace=True)
        )
        self.ca = ChannelAttention(64)
        self.sa = SpatialAttention()
        self.sigmoid_spatial = nn.Sigmoid()
    
    def forward(self, x):
        res1 = x
        res2 = x
        x = self.conv1(x)        
        x = x + self.conv2(x)
        x = self.conv3(x)
        x = x + self.conv4(x)
        x = self.conv5(x)
        x = x + self.conv6(x)
        x = self.conv7(x)
        x_mask = self.sigmoid_spatial(x)
        res1 = res1 * x_mask
        return res2 + res1

              
class GLSA(nn.Module):

    def __init__(self, input_dim=512, embed_dim=32, k_s=3):
        super().__init__()
                      
        self.conv1_1 = BasicConv2d(embed_dim*2,embed_dim, 1)
        self.conv1_1_1 = BasicConv2d(input_dim//2,embed_dim,1)
        self.local_11conv = nn.Conv2d(input_dim//2,embed_dim,1)
        self.global_11conv = nn.Conv2d(input_dim//2,embed_dim,1)
        self.GlobelBlock = ContextBlock(inplanes= embed_dim, ratio=2)
        self.local = ConvBranch(in_features = embed_dim, hidden_features = embed_dim, out_features = embed_dim)

    def forward(self, x):
        b, c, h, w = x.size()
        x_0, x_1 = x.chunk(2,dim = 1)  
        
    # local block 
        local = self.local(self.local_11conv(x_0))
        
    # Globel block    
        Globel = self.GlobelBlock(self.global_11conv(x_1))

    # concat Globel + local
        x = torch.cat([local,Globel], dim=1)
        x = self.conv1_1(x)

        return x    

class SBA(nn.Module):

    def __init__(self,input_dim = 64):
        super().__init__()

        self.input_dim = input_dim

        self.d_in1 = BasicConv2d(input_dim//2, input_dim//2, 1)
        self.d_in2 = BasicConv2d(input_dim//2, input_dim//2, 1)       
                

        self.conv = nn.Sequential(BasicConv2d(input_dim, input_dim, 3,1,1), nn.Conv2d(input_dim, 1, kernel_size=1, bias=False))
        self.fc1 = nn.Conv2d(input_dim, input_dim//2, kernel_size=1, bias=False)
        self.fc2 = nn.Conv2d(input_dim, input_dim//2, kernel_size=1, bias=False)
        
        self.Sigmoid = nn.Sigmoid()
        
    def forward(self, H_feature, L_feature):

        L_feature = self.fc1(L_feature)
        H_feature = self.fc2(H_feature)
        
        g_L_feature =  self.Sigmoid(L_feature)
        g_H_feature = self.Sigmoid(H_feature)
        
        L_feature = self.d_in1(L_feature)
        H_feature = self.d_in2(H_feature)


        L_feature = L_feature + L_feature * g_L_feature + (1 - g_L_feature) * Upsample(g_H_feature * H_feature, size= L_feature.size()[2:], align_corners=False)
        H_feature = H_feature + H_feature * g_H_feature + (1 - g_H_feature) * Upsample(g_L_feature * L_feature, size= H_feature.size()[2:], align_corners=False) 
        
        H_feature = Upsample(H_feature, size = L_feature.size()[2:])
        out = self.conv(torch.cat([H_feature,L_feature], dim=1))
        return out
        
            
class DuAT(nn.Module):
    def __init__(self, dim=32, dims= [64, 128, 320, 512], learning_rate=None,  loss_fn=None, optimizer=None, device=None, weight_decay=None):
        super(DuAT, self).__init__()

                    
        if (device is None):
            self.device = DEVICE
        else:
            self.device = device

        if (learning_rate is None):
            self.learning_rate = 1e-5
        else:
            self.learning_rate = learning_rate
        
        if (weight_decay is None):
            self.weight_decay = 1e-5
        else:
            self.weight_decay = weight_decay

        self.dims = dims

        self.pad = False
        self.backbone = pvt_v2_b2()  # [64, 128, 320, 512]
        
        
        c1_in_channels, c2_in_channels, c3_in_channels, c4_in_channels = dims[0], dims[1], dims[2], dims[3]
        

        self.embed4 = nn.Embedding(48, c4_in_channels)
        self.embed3 = nn.Embedding(96, c3_in_channels)
        self.embed2 = nn.Embedding(192, c2_in_channels)
        self.embed1 = nn.Embedding(384, c1_in_channels)

        self.bn4 = nn.BatchNorm2d(c4_in_channels)
        self.bn3 = nn.BatchNorm2d(c3_in_channels)
        self.bn2 = nn.BatchNorm2d(c2_in_channels)
        self.bn1 = nn.BatchNorm2d(c1_in_channels)

        self.GLSA_c4 = GLSA(input_dim=c4_in_channels, embed_dim=dim)
        self.GLSA_c3 = GLSA(input_dim=c3_in_channels, embed_dim=dim)
        self.GLSA_c2 = GLSA(input_dim=c2_in_channels, embed_dim=dim)
        self.L_feature = BasicConv2d(c1_in_channels, dim, 3,1,1)
        
        self.SBA = SBA(input_dim = dim)
        self.fuse = BasicConv2d(dim * 2, dim, 1)
        self.fuse2 = nn.Sequential(BasicConv2d(dim*3, dim, 1,1),nn.BatchNorm2d(dim),nn.Conv2d(dim, 1, kernel_size=1, bias=False))

        

        if (loss_fn is None):
            self.loss_fn = structure_loss
        else:
            self.loss_fn = loss_fn

        if (optimizer is None):
            self.optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        else:
            self.optimizer = optimizer(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)

        self = self.to(self.device)

    def forward(self, x, mask):
        # backbone
        x = x.to(self.device)
        c1, c2, c3, c4 = self.backbone(x)
        
        n, _, h, w = c4.shape        
        _c4 = self.GLSA_c4(c4) # [1, 64, 11, 11]
        _c4 = Upsample(_c4, c3.size()[2:])
        _c3 = self.GLSA_c3(c3) # [1, 64, 22, 22]
        _c2 = self.GLSA_c2(c2) # [1, 64, 44, 44]
        
        output = torch.cat([Upsample(_c4, c2.size()[2:]), Upsample(_c3, c2.size()[2:]), _c2], dim=1)
        
        L_feature = self.L_feature(c1)  # [1, 64, 88, 88]
        H_feature = self.fuse(torch.cat([_c4, _c3], dim=1))
        H_feature = Upsample(H_feature,c2.size()[2:])
        
        output2 = self.SBA(H_feature,L_feature)
        h = x.shape[2] // 4
        output = F.interpolate(output, scale_factor=8, mode='bilinear')
        # output = torch.cat([output, Upsample(c1, output.size()[2:])], dim=1)
        output = self.fuse2(output)
        # return torch.sigmoid(output)
        output2 = F.interpolate(output2, scale_factor=4, mode='bilinear')
        if (self.pad):
            output = output[:,:,h:-h,:]
            output2 = output2[:,:,h:-h,:]
        
        return output, output2
        # return F.sigmoid(output), F.sigmoid(output2)
        
    def fit(self, X, mask, y):
        X, mask, y = X.to(self.device), mask.to(self.device), y.to(self.device)
        h1, h2 = self(X, mask)
        self.optimizer.zero_grad()
        loss = self.loss_fn(h1, y) + self.loss_fn(h2, y)
        loss.backward()
        self.optimizer.step()
        loss = loss.item()
        pred = (h2 + h1 > 0).int()
        numt = torch.sum(mask)
        TP = torch.sum(torch.minimum(y, pred)).item() / numt
        TN = torch.sum(torch.minimum(1-y, 1-pred) * mask).item() / numt
        FN = torch.sum(torch.minimum(y, 1-pred)).item() / numt
        FP = torch.sum(torch.minimum(1-y, pred)).item() / numt
        if (torch.any(TP < 0) or torch.any(TN < 0) or torch.any(FN < 0) or torch.any(FP < 0)):
            raise AssertionError()
        return (loss, torch.tensor([TP, FP, FN, TN]))

    def test(self, X, mask, y):
        X, mask, y = X.to(self.device), mask.to(self.device), y.to(self.device)
        h1, h2 = self(X, mask)
        loss = self.loss_fn(h1, y) + self.loss_fn(h2, y)
        loss = loss.item()
        pred = (h2 + h1 > 0).int()
        numt = torch.sum(mask)
        TP = torch.sum(torch.minimum(y, pred)).item() / numt
        TN = torch.sum(torch.minimum(1-y, 1-pred) * mask).item() / numt
        FN = torch.sum(torch.minimum(y, 1-pred)).item() / numt
        FP = torch.sum(torch.minimum(1-y, pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))


In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from lib.pvtv2 import pvt_v2_b2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
# from mmcv.cnn import ConvModule
from torch.nn import Conv2d, UpsamplingBilinear2d
import warnings
import torch
# from mmcv.cnn import constant_init, kaiming_init
from torch import nn
from torchvision.transforms.functional import normalize
from PVTv2.pvtv2 import pvt_v2_b2
warnings.filterwarnings('ignore')

class SpatBlock(nn.Module):
    def __init__(self, dim=32, order=5):
        super().__init__()
        self.dim = dim
        self.order = order
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.blocks = nn.ModuleList([nn.Conv2d(1, dim, 1)] + [nn.Conv2d(1, dim, 1, bias=False) for _ in range(order-1)])

        self = self.to(self.device)

    def forward(self, x):
        B, C, H, W = x.shape

        pos = torch.arange(W) / W
        pos = torch.reshape(pos, (1, 1, 1, W)).to(self.device)
        
        func = torch.zeros_like(x)

        for i, block in enumerate(self.blocks):
            func += block(pos**(i+1))

        out = F.tanh(func) * x
        
        return out
        

def structure_loss(pred, mask):
    weit = 1 + 5 * torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wbce = F.binary_cross_entropy_with_logits(pred, mask, reduce='none')
    wbce = (weit * wbce).sum(dim=(2, 3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask) * weit).sum(dim=(2, 3))
    union = ((pred + mask) * weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1) / (union - inter + 1)

    return (wbce + wiou).mean()

# def structure_loss(pred, mask):
#     pred = torch.sigmoid(pred)
#     loss1 = F.binary_cross_entropy(pred, mask)
#     tp = pred * mask
#     fp = pred * (1 - mask)
#     fn = (1 - pred) * mask
#     tp = torch.mean(tp, dim=0)
#     fp = torch.mean(fp, dim=0)
#     fn = torch.mean(fn, dim=0)
#     iou = tp / (tp + fp + fn)
#     iou = torch.mean(iou)
#     loss2 = 1 - iou
#     return loss1 + loss2

class BasicConv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1):
        super(BasicConv2d, self).__init__()

        self.conv = nn.Conv2d(in_planes, out_planes,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False)
        self.bn = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
        
class Block(nn.Sequential):
    def __init__(self, input_num, num1, num2, dilation_rate, drop_out, bn_start=True, norm_layer=nn.BatchNorm2d):
        super(Block, self).__init__()
        if bn_start:
            self.add_module('norm1', norm_layer(input_num)),

        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(in_channels=input_num, out_channels=num1, kernel_size=1)),

        self.add_module('norm2', norm_layer(num1)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(in_channels=num1, out_channels=num2, kernel_size=3,
                                            dilation=dilation_rate, padding=dilation_rate)),
        self.drop_rate = drop_out

    def forward(self, _input):
        feature = super(Block, self).forward(_input)
        if self.drop_rate > 0:
            feature = F.dropout2d(feature, p=self.drop_rate, training=self.training)
        return feature


def Upsample(x, size, align_corners = False):
    """
    Wrapper Around the Upsample Call
    """
    return nn.functional.interpolate(x, size=size, mode='bilinear', align_corners=align_corners)

    
# def last_zero_init(m):
#     if isinstance(m, nn.Sequential):
#         constant_init(m[-1], val=0)
#     else:
#         constant_init(m, val=0)


class ContextBlock(nn.Module):

    def __init__(self,
                 inplanes,
                 ratio,
                 pooling_type='att',
                 fusion_types=('channel_mul', )):
        super(ContextBlock, self).__init__()
        assert pooling_type in ['avg', 'att']
        assert isinstance(fusion_types, (list, tuple))
        valid_fusion_types = ['channel_add', 'channel_mul']
        assert all([f in valid_fusion_types for f in fusion_types])
        assert len(fusion_types) > 0, 'at least one fusion should be used'
        self.inplanes = inplanes
        self.ratio = ratio
        self.planes = int(inplanes * ratio)
        self.pooling_type = pooling_type
        self.fusion_types = fusion_types
        if pooling_type == 'att':
            self.conv_mask = nn.Conv2d(inplanes, 1, kernel_size=1)
            self.softmax = nn.Softmax(dim=2)
        else:
            self.avg_pool = nn.AdaptiveAvgPool2d(1)
        if 'channel_add' in fusion_types:
            self.channel_add_conv = nn.Sequential(
                nn.Conv2d(self.inplanes, self.planes, kernel_size=1),
                nn.LayerNorm([self.planes, 1, 1]),
                nn.ReLU(inplace=True),  # yapf: disable
                nn.Conv2d(self.planes, self.inplanes, kernel_size=1))
        else:
            self.channel_add_conv = None
        if 'channel_mul' in fusion_types:
            self.channel_mul_conv = nn.Sequential(
                nn.Conv2d(self.inplanes, self.planes, kernel_size=1),
                nn.LayerNorm([self.planes, 1, 1]),
                nn.ReLU(inplace=True),  # yapf: disable
                nn.Conv2d(self.planes, self.inplanes, kernel_size=1))
        else:
            self.channel_mul_conv = None

    def spatial_pool(self, x):
        batch, channel, height, width = x.size()
        if self.pooling_type == 'att':
            input_x = x
            # [N, C, H * W]
            input_x = input_x.view(batch, channel, height * width)
            # [N, 1, C, H * W]
            input_x = input_x.unsqueeze(1)
            # [N, 1, H, W]
            context_mask = self.conv_mask(x)
            # [N, 1, H * W]
            context_mask = context_mask.view(batch, 1, height * width)
            # [N, 1, H * W]
            context_mask = self.softmax(context_mask)
            # [N, 1, H * W, 1]
            context_mask = context_mask.unsqueeze(-1)
            # [N, 1, C, 1]
            context = torch.matmul(input_x, context_mask)
            # [N, C, 1, 1]
            context = context.view(batch, channel, 1, 1)
        else:
            # [N, C, 1, 1]
            context = self.avg_pool(x)

        return context

    def forward(self, x):
        # [N, C, 1, 1]
        context = self.spatial_pool(x)

        out = x
        if self.channel_mul_conv is not None:
            # [N, C, 1, 1]
            channel_mul_term = torch.sigmoid(self.channel_mul_conv(context))
            out = out + out * channel_mul_term
        if self.channel_add_conv is not None:
            # [N, C, 1, 1]
            channel_add_term = self.channel_add_conv(context)
            out = out + channel_add_term

        return out



class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1   = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)


class ConvBranch(nn.Module):
    def __init__(self, in_features, hidden_features = None, out_features = None):
        super().__init__()
        hidden_features = hidden_features or in_features
        out_features = out_features or in_features
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_features, hidden_features, 1, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 3, padding=1, groups=hidden_features, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 1, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 3, padding=1, groups=hidden_features, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 1, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.SiLU(inplace=True)
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(hidden_features, hidden_features, 3, padding=1, groups=hidden_features, bias=False),
            nn.BatchNorm2d(hidden_features),
            nn.ReLU(inplace=True)
        )
        self.conv7 = nn.Sequential(
            nn.Conv2d(hidden_features, out_features, 1, bias=False),
            nn.ReLU(inplace=True)
        )
        self.ca = ChannelAttention(64)
        self.sa = SpatialAttention()
        self.sigmoid_spatial = nn.Sigmoid()
    
    def forward(self, x):
        res1 = x
        res2 = x
        x = self.conv1(x)        
        x = x + self.conv2(x)
        x = self.conv3(x)
        x = x + self.conv4(x)
        x = self.conv5(x)
        x = x + self.conv6(x)
        x = self.conv7(x)
        x_mask = self.sigmoid_spatial(x)
        res1 = res1 * x_mask
        return res2 + res1

              
class GLSA(nn.Module):

    def __init__(self, input_dim=512, embed_dim=32, k_s=3):
        super().__init__()
                      
        self.conv1_1 = BasicConv2d(embed_dim*2,embed_dim, 1)
        self.conv1_1_1 = BasicConv2d(input_dim//2,embed_dim,1)
        self.local_11conv = nn.Conv2d(input_dim//2,embed_dim,1)
        self.global_11conv = nn.Conv2d(input_dim//2,embed_dim,1)
        self.GlobelBlock = ContextBlock(inplanes= embed_dim, ratio=2)
        self.local = ConvBranch(in_features = embed_dim, hidden_features = embed_dim, out_features = embed_dim)

    def forward(self, x):
        b, c, h, w = x.size()
        x_0, x_1 = x.chunk(2,dim = 1)  
        
    # local block 
        local = self.local(self.local_11conv(x_0))
        
    # Globel block    
        Globel = self.GlobelBlock(self.global_11conv(x_1))

    # concat Globel + local
        x = torch.cat([local,Globel], dim=1)
        x = self.conv1_1(x)

        return x    

class SBA(nn.Module):

    def __init__(self,input_dim = 64):
        super().__init__()

        self.input_dim = input_dim

        self.d_in1 = BasicConv2d(input_dim//2, input_dim//2, 1)
        self.d_in2 = BasicConv2d(input_dim//2, input_dim//2, 1)       
                

        self.conv = nn.Sequential(BasicConv2d(input_dim, input_dim, 3,1,1), nn.Conv2d(input_dim, 1, kernel_size=1, bias=False))
        self.fc1 = nn.Conv2d(input_dim, input_dim//2, kernel_size=1, bias=False)
        self.fc2 = nn.Conv2d(input_dim, input_dim//2, kernel_size=1, bias=False)
        
        self.Sigmoid = nn.Sigmoid()
        
    def forward(self, H_feature, L_feature):

        L_feature = self.fc1(L_feature)
        H_feature = self.fc2(H_feature)
        
        g_L_feature =  self.Sigmoid(L_feature)
        g_H_feature = self.Sigmoid(H_feature)
        
        L_feature = self.d_in1(L_feature)
        H_feature = self.d_in2(H_feature)


        L_feature = L_feature + L_feature * g_L_feature + (1 - g_L_feature) * Upsample(g_H_feature * H_feature, size= L_feature.size()[2:], align_corners=False)
        H_feature = H_feature + H_feature * g_H_feature + (1 - g_H_feature) * Upsample(g_L_feature * L_feature, size= H_feature.size()[2:], align_corners=False) 
        
        H_feature = Upsample(H_feature, size = L_feature.size()[2:])
        out = self.conv(torch.cat([H_feature,L_feature], dim=1))
        return out
        
            
class DuAT(nn.Module):
    def __init__(self, dim=32, dims= [64, 128, 320, 512], learning_rate=None,  loss_fn=None, optimizer=None, device=None, weight_decay=None):
        super(DuAT, self).__init__()

                    
        if (device is None):
            self.device = DEVICE
        else:
            self.device = device

        if (learning_rate is None):
            self.learning_rate = 1e-5
        else:
            self.learning_rate = learning_rate
        
        if (weight_decay is None):
            self.weight_decay = 1e-5
        else:
            self.weight_decay = weight_decay

        self.dims = dims

        # self.pad = False
        self.backbone = pvt_v2_b2()  # [64, 128, 320, 512]
        
        c1_in_channels, c2_in_channels, c3_in_channels, c4_in_channels = dims[0], dims[1], dims[2], dims[3]

        self.shuffle4 = nn.Conv2d(c4_in_channels, c4_in_channels, 1)
        self.shuffle3 = nn.Conv2d(c3_in_channels, c3_in_channels, 1)
        self.shuffle2 = nn.Conv2d(c2_in_channels, c2_in_channels, 1)
        self.shuffle1 = nn.Conv2d(c1_in_channels, c1_in_channels, 1)

        self.bn4 = nn.BatchNorm2d(c4_in_channels)
        self.bn3 = nn.BatchNorm2d(c3_in_channels)
        self.bn2 = nn.BatchNorm2d(c2_in_channels)
        self.bn1 = nn.BatchNorm2d(c1_in_channels)

        self.embed4 = SpatBlock(dim=c4_in_channels)
        self.embed3 = SpatBlock(dim=c3_in_channels)
        self.embed2 = SpatBlock(dim=c2_in_channels)
        self.embed1 = SpatBlock(dim=c1_in_channels)

        self.bn4 = nn.BatchNorm2d(c4_in_channels)
        self.bn3 = nn.BatchNorm2d(c3_in_channels)
        self.bn2 = nn.BatchNorm2d(c2_in_channels)
        self.bn1 = nn.BatchNorm2d(c1_in_channels)

        self.GLSA_c4 = GLSA(input_dim=c4_in_channels, embed_dim=dim)
        self.GLSA_c3 = GLSA(input_dim=c3_in_channels, embed_dim=dim)
        self.GLSA_c2 = GLSA(input_dim=c2_in_channels, embed_dim=dim)
        self.L_feature = BasicConv2d(c1_in_channels, dim, 3,1,1)
        
        self.SBA = SBA(input_dim = dim)
        self.fuse = BasicConv2d(dim * 2, dim, 1)
        self.fuse2 = nn.Sequential(BasicConv2d(dim*3, dim, 1,1),nn.BatchNorm2d(dim),nn.Conv2d(dim, 1, kernel_size=1, bias=False))

        

        if (loss_fn is None):
            self.loss_fn = structure_loss
        else:
            self.loss_fn = loss_fn

        if (optimizer is None):
            self.optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        else:
            self.optimizer = optimizer(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)

        self = self.to(self.device)

    def forward(self, x, mask):
        # backbone
        x = x.to(self.device)
        c1, c2, c3, c4 = self.backbone(x)


        
        # c1 = self.bn1(self.shuffle1(c1))
        # c2 = self.bn2(self.shuffle2(c2))
        # c3 = self.bn3(self.shuffle3(c3))
        # c4 = self.bn4(self.shuffle4(c4))
        
        # c1 = torch.cat([c1,
        #                 torch.arange(c1.shape[2], dtype=c1.dtype, device=self.device).view(1,1,-1,1).repeat(c1.shape[0], 1, 1, c1.shape[3]) / c1.shape[2],
        #                 torch.arange(c1.shape[3], dtype=c1.dtype, device=self.device).view(1,1,1,-1).repeat(c1.shape[0], 1, c1.shape[2], 1) / c1.shape[3]], dim=1)
        # c2 = torch.cat([c2,
        #                 torch.arange(c2.shape[2], dtype=c2.dtype, device=self.device).view(1,1,-1,1).repeat(c2.shape[0], 1, 1, c2.shape[3]) / c2.shape[2],
        #                 torch.arange(c2.shape[3], dtype=c2.dtype, device=self.device).view(1,1,1,-1).repeat(c2.shape[0], 1, c2.shape[2], 1) / c2.shape[3]], dim=1)
        # c3 = torch.cat([c3,
        #                 torch.arange(c3.shape[2], dtype=c3.dtype, device=self.device).view(1,1,-1,1).repeat(c1.shape[0], 1, 1, c3.shape[3]) / c3.shape[2],
        #                 torch.arange(c3.shape[3], dtype=c3.dtype, device=self.device).view(1,1,1,-1).repeat(c1.shape[0], 1, c3.shape[2], 1) / c3.shape[3]], dim=1)
        # c4 = torch.cat([c4,
        #                 torch.arange(c4.shape[2], dtype=c4.dtype, device=self.device).view(1,1,-1,1).repeat(c4.shape[0], 1, 1, c4.shape[3]) / c4.shape[2],
        #                 torch.arange(c4.shape[3], dtype=c4.dtype, device=self.device).view(1,1,1,-1).repeat(c4.shape[0], 1, c4.shape[2], 1) / c4.shape[3]], dim=1)

        c1 = self.bn1(self.embed1(c1))
        c2 = self.bn2(self.embed2(c2))
        c3 = self.bn3(self.embed3(c3))
        c4 = self.bn4(self.embed4(c4))
        
        n, _, h, w = c4.shape        
        _c4 = self.GLSA_c4(c4) # [1, 64, 11, 11]
        _c4 = Upsample(_c4, c3.size()[2:])
        _c3 = self.GLSA_c3(c3) # [1, 64, 22, 22]
        _c2 = self.GLSA_c2(c2) # [1, 64, 44, 44]
        
        output = torch.cat([Upsample(_c4, c2.size()[2:]), Upsample(_c3, c2.size()[2:]), _c2], dim=1)
        
        L_feature = self.L_feature(c1)  # [1, 64, 88, 88]
        H_feature = self.fuse(torch.cat([_c4, _c3], dim=1))
        H_feature = Upsample(H_feature,c2.size()[2:])
        
        output2 = self.SBA(H_feature,L_feature)
        h = x.shape[2] // 4
        output = F.interpolate(output, scale_factor=8, mode='bicubic')
        # output = torch.cat([output, Upsample(c1, output.size()[2:])], dim=1)
        output = self.fuse2(output)
        # return torch.sigmoid(output)
        output2 = F.interpolate(output2, scale_factor=4, mode='bicubic')

        
        return output, output2
        # return F.sigmoid(output[:,:,h:-h,:])
        
    def fit(self, X, mask, y):
        X, mask, y = X.to(self.device), mask.to(self.device), y.to(self.device)
        h1, h2 = self(X, mask)
        self.optimizer.zero_grad()
        loss = self.loss_fn(h1, y) + self.loss_fn(h2, y)
        loss.backward()
        self.optimizer.step()
        loss = loss.item()
        pred = (h2 > 0).int()
        numt = torch.sum(mask)
        TP = torch.sum(torch.minimum(y, pred)).item() / numt
        TN = torch.sum(torch.minimum(1-y, 1-pred) * mask).item() / numt
        FN = torch.sum(torch.minimum(y, 1-pred)).item() / numt
        FP = torch.sum(torch.minimum(1-y, pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))

    def test(self, X, mask, y):
        X, mask, y = X.to(self.device), mask.to(self.device), y.to(self.device)
        h1, h2 = self(X, mask)
        loss = self.loss_fn(h1, y) + self.loss_fn(h2, y)
        loss = loss.item()
        pred = (h2 > 0).int()
        numt = torch.sum(mask)
        TP = torch.sum(torch.minimum(y, pred)).item() / numt
        TN = torch.sum(torch.minimum(1-y, 1-pred) * mask).item() / numt
        FN = torch.sum(torch.minimum(y, 1-pred)).item() / numt
        FP = torch.sum(torch.minimum(1-y, pred)).item() / numt
        return (loss, torch.tensor([TP, FP, FN, TN]))


# Cartesian test

In [14]:

def process_tensor_cart(tensor):
    tensor_np = tensor
    kernel = np.ones((3,3),np.uint8)
    closing = cv.morphologyEx(tensor_np, cv.MORPH_ERODE, kernel)
    cleaned = closing
    cleaned = cv.morphologyEx(cleaned, cv.MORPH_DILATE, kernel)
    
    kernel = np.ones((10, 10),np.uint8)

    cleaned = cv.morphologyEx(cleaned, cv.MORPH_CLOSE, kernel)
    return cleaned
    
process_tensor = process_tensor_cart

In [18]:
results = []

def output(model):
    num_batches = len(test_dl)
    model.eval()
    with torch.no_grad():                
        num_batches = len(test_dl)
        dices = []
        with tqdm(total=num_batches) as pbar:
            pbar.set_description("Validation Dataset")
            for (X, mask_l, mask_s, y, filename, center) in test_dl:
                X = X.to(DEVICE)
                mask_s = mask_s.to(DEVICE)
                h1, h2 = model(X, mask_s)
                h = h1 + h2
                h = h.detach().cpu().numpy()[0,0]
                h = cv.resize(h, (512, 384))
                h = np.uint8(h > .0)
                h = process_tensor(h)
                y_tilde = np.round(h)
                y = to_cart(y.detach().cpu().numpy()[0,0], (int(center[0].item() * 512), int(center[1].item() * 384)))
                y = np.round(y)
                # print(type(y), type(y_tilde))
                tp = np.sum(np.minimum(y_tilde, y))
                fp = np.sum(np.minimum(y_tilde, 1 - y))
                fn = np.sum(np.minimum(1 - y_tilde, y))
                tn = np.sum(np.minimum(1 - y_tilde, 1 - y))
                dice = 2 * tp / (2 * tp + fp + fn)
                dices.append(dice)
                # img = np.concatenate([y_tilde * 255, y * 255], axis=1)
                # cv.imwrite(os.path.join('Final-Improv', f'{dice:.4f}-' + filename[0] + '.jpg'), img)
                pbar.update(1)
        print(np.mean(np.array(dices)))
        results.append(np.mean(np.array(dices)))

In [19]:
def raw_output(model):
    num_batches = len(test_dl)
    model.eval()
    with torch.no_grad():                
        num_batches = len(test_dl)
        dices = []
        with tqdm(total=num_batches) as pbar:
            pbar.set_description("Test Dataset")
            for (X, mask_l, mask_s, y, filename, center) in test_dl:
                X = X.to(DEVICE)
                mask_s = mask_s.to(DEVICE)
                h1, h2 = model(X, mask_s)
                h = h1 + h2
                h = h.detach().cpu().numpy()[0,0]
                h = cv.resize(h, (512, 384))
                h = np.uint8(h > .0)
                y_tilde = np.round(h)
                y = y.detach().cpu().numpy()[0,0]
                y = np.round(y)
                # print(type(y), type(y_tilde))
                tp = np.sum(np.minimum(y_tilde, y))
                fp = np.sum(np.minimum(y_tilde, 1 - y))
                fn = np.sum(np.minimum(1 - y_tilde, y))
                tn = np.sum(np.minimum(1 - y_tilde, 1 - y))
                dice = 2 * tp / (2 * tp + fp + fn)
                dices.append(dice)
                # img = np.concatenate([y_tilde * 255, y * 255], axis=1)
                # cv.imwrite(os.path.join('Final-Improv', f'{dice:.4f}-' + filename[0] + '.jpg'), img)
                pbar.update(1)
        print(np.mean(np.array(dices)))
        results_pol.append(np.mean(np.array(dices)))

# Training

In [33]:
results_pol = []
results = []
model_name = 'Cart-FocalLoss'
for iter in range(1,6):
    print(f'Run {iter} ===========')
    net = DuAT(dim=32, optimizer=torch.optim.AdamW, learning_rate=1e-4, weight_decay=3e-4, loss_fn=FocalLoss(1))
    maxdice = 0.
    
    for t in range(30):
        torch.cuda.empty_cache()
        print(f"Epoch {t+1} ---------------------")
        print(f"Training Set -----")
        train_loss, _ = train(net)
        print(f"Validation Set -----")
        test_loss, dice = test(net)
        print(f"\rDice: {dice: .4f}, Loss: {test_loss : .4f}")
        if dice > maxdice:
            maxdice = dice
            torch.save(net, f'FinalModels/{model_name}-{iter}.pth')
            
    print(f"Test Set -----")
    net = torch.load(f'FinalModels/{model_name}-{iter}.pth')
    output(net)
    raw_output(net)

print('FINAL RESULTS:')
print(f'CARTESIAN={np.mean(np.array(results_pol)):.4f}+/-{np.std(np.array(results_pol)):.4f}')

Run 1 ===========
Epoch 1 ---------------------
Training Set -----


Avg. Loss: 0.3366, Avg. Accuracy: 0.9073, Avg. Dice: 0.7793: 100%|███████████████████| 519/519 [15:44<00:00,  1.82s/it]


Validation Set -----


Avg. Loss: 0.2166, Avg. Accuracy: 0.9380, Avg. Dice: 0.8319: 100%|█████████████████████| 65/65 [01:27<00:00,  1.35s/it]


Dice:  0.8319, Loss:  0.2166
Epoch 2 ---------------------
Training Set -----


...

Dice:  0.8523, Loss:  0.2315
Test Set -----


Validation Dataset: 100%|████████████████████████████████████████████████████████████| 259/259 [01:26<00:00,  2.99it/s]


0.1598851496285095


Test Dataset: 100%|██████████████████████████████████████████████████████████████████| 259/259 [01:24<00:00,  3.08it/s]

0.8779404137439657
FINAL RESULTS:
POLAR=0.8777+/-0.0012
POLAR=0.1688+/-0.0052
